In [9]:
from typing import Annotated, Optional, TypedDict
from pydantic import BaseModel, Field
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage,ToolMessage
from langgraph.graph import StateGraph, END,add_messages
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
from langchain_community.tools.tavily_search import TavilySearchResults
from langgraph.checkpoint.memory import MemorySaver
from uuid import uuid4
load_dotenv()

True

In [10]:
llm_model = ChatOpenAI()
search_tool = TavilySearchResults()
memory = MemorySaver()

tools = [search_tool]

llm_with_tools = llm_model.bind_tools(tools)

In [11]:
class Agent_state(TypedDict):
   messages : Annotated[list, add_messages]

In [12]:
async def model_node(state:Agent_state):
    result = await llm_with_tools.ainvoke(state['messages'])
    return {
        "messages":[result]
    }

In [13]:
async def tool_router(state:Agent_state):
    last_message = state["messages"][-1]
    if(hasattr(last_message, "tool_calls") and len(last_message.tool_calls) > 0):
        return "tool_node"
    else: 
        return END
    
async def tool_node(state:Agent_state):
    tool_calls = state["messages"][-1].tool_calls
    print(tool_calls)

    tool_messages = []

    for tool_call in tool_calls:
        tool_name = tool_call['name']
        tool_args = tool_call["args"]
        tool_id = tool_call["id"]

        if tool_name == "tavily_search_results_json":
            # Invoke the search tool with the arguments
            search_result = await search_tool.ainvoke(tool_args)
            # Create a message for the tool response
            tool_message = ToolMessage(
                content = str(search_result),
                tool_call_id = tool_id,
                tool_name = tool_name,
            )
    
            tool_messages.append(tool_message)
    
    return {
        'messages':tool_messages
    }

In [14]:
graph_builder = StateGraph(Agent_state)
graph_builder.add_node("model_node", model_node)
graph_builder.add_node("tool_node", tool_node)
graph_builder.set_entry_point("model_node")
graph_builder.add_conditional_edges("model_node", tool_router)
graph_builder.add_edge("tool_node", "model_node")

graph = graph_builder.compile(checkpointer=memory)

config = {
    "configurable":{
        "thread_id":2
    }
}

# response = await graph.ainvoke({"messages": ["when is the next spaceX Launch ?"]}, config=config)

In [15]:
events = graph.astream_events(
    {"messages":["when is the next spaceX Launch ?"]},
    config=config,
)

async for event in events:
    # print(event)
    if event['event'] == "on_chat_model_stream":
        print(event['data']['chunk'].content, end='')

[{'name': 'tavily_search_results_json', 'args': {'query': 'next SpaceX launch'}, 'id': 'call_KFrS8JowDuQGew7tWj8FZgtb', 'type': 'tool_call'}]
The next SpaceX launch is scheduled for June 13 at 1:46 am UTC. It will be the Falcon 9 Block 5 Starlink Group 15-6 mission. You can find more information on the [RocketLaunch.org website](https://rocketlaunch.org/launch-schedule/spacex).